In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras import regularizers, optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Using TensorFlow backend.


In [2]:
from cs231n.data_utils import load_CIFAR10
from sklearn.decomposition import PCA

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, is_pca=False):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Apply PCA
    if is_pca == True:
        pca = PCA(n_components=32*32)
        X_train = pca.fit_transform(X_train.reshape((X_train.shape[0], -1)))
        X_test = pca.fit_transform(X_test.reshape((X_test.shape[0], -1)))
        
    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    
    # Range should be [-1, 1]
    X_train = 2 * X_train - 1
    X_val = 2 * X_val - 1
    X_test = 2 * X_test - 1

    # Reshape data to rows
    X_train = X_train.reshape(num_training, -1)
    X_val = X_val.reshape(num_validation, -1)
    X_test = X_test.reshape(num_test, -1)

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data(is_pca=False)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 3072)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3072)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3072)
Test labels shape:  (1000,)


In [3]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, num_classes=None)
y_val = to_categorical(y_val, num_classes=None)


In [8]:
best_net = None # store the best model into this 

# learning_rates = [1.8e-3]
learning_rates = [1e-2]
regularization_strengths = [1e-1]

results = {}
best_val = -1

input_size = 32 * 32 * 3
hidden_sizes = [400]
num_classes = 10

def generate_arrays(X, y, batch_size):
    num_train = X.shape[0]
    while 1:
        mask = np.random.choice(num_train, batch_size, replace=True)
        X_batch = X[mask]
        y_batch = y[mask]
        
        yield (X_batch, y_batch)

for learning_rate in learning_rates:
    for regularization_strength in regularization_strengths:
        for hidden_size in hidden_sizes:
            model = Sequential()
            model.add(Dense(hidden_size, activation='relu', kernel_initializer='uniform',
                            kernel_regularizer=regularizers.l2(regularization_strength),
                            input_shape=(input_size,)))
                      
            model.add(Dense(num_classes, activation='softmax',
                            kernel_regularizer=regularizers.l2(regularization_strength),))
            
            optimizer = optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=False)
        
            model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

            
            loss_hist = model.fit_generator(generate_arrays(X_train, y_train, batch_size=200),
                                            validation_data=(X_val, y_val),
                                            steps_per_epoch=30,
                                            epochs=100)

Epoch 1/100
30/30 [==============================] - 0s - loss: 93.7876 - acc: 0.1523 - val_loss: 59.7727 - val_acc: 0.1560
Epoch 2/100
30/30 [==============================] - 0s - loss: 39.2155 - acc: 0.1487 - val_loss: 24.8295 - val_acc: 0.1360
Epoch 3/100
30/30 [==============================] - 0s - loss: 21.7665 - acc: 0.1218 - val_loss: 20.2732 - val_acc: 0.0780
Epoch 4/100
30/30 [==============================] - 0s - loss: 18.0558 - acc: 0.1037 - val_loss: 16.7105 - val_acc: 0.0780
Epoch 5/100
30/30 [==============================] - 0s - loss: 15.4913 - acc: 0.1007 - val_loss: 15.2738 - val_acc: 0.0780
Epoch 6/100
30/30 [==============================] - 0s - loss: 14.1213 - acc: 0.1007 - val_loss: 12.4770 - val_acc: 0.1260
Epoch 7/100
30/30 [==============================] - 0s - loss: 17.9515 - acc: 0.1177 - val_loss: 19.7709 - val_acc: 0.1330
Epoch 8/100
30/30 [==============================] - 0s - loss: 18.5303 - acc: 0.1170 - val_loss: 16.9408 - val_acc: 0.1460
Epoch 9/

30/30 [==============================] - 0s - loss: 18.5356 - acc: 0.1197 - val_loss: 21.3482 - val_acc: 0.0850
Epoch 67/100
30/30 [==============================] - 0s - loss: 21.0002 - acc: 0.0937 - val_loss: 18.9557 - val_acc: 0.0870
Epoch 68/100
30/30 [==============================] - 0s - loss: 16.9988 - acc: 0.0937 - val_loss: 15.7548 - val_acc: 0.0870
Epoch 69/100
30/30 [==============================] - 0s - loss: 15.0336 - acc: 0.0993 - val_loss: 14.6987 - val_acc: 0.0850
Epoch 70/100
30/30 [==============================] - 0s - loss: 16.4531 - acc: 0.0977 - val_loss: 17.4435 - val_acc: 0.1070
Epoch 71/100
30/30 [==============================] - 0s - loss: 17.4831 - acc: 0.0938 - val_loss: 18.2872 - val_acc: 0.0980
Epoch 72/100
30/30 [==============================] - 0s - loss: 17.2870 - acc: 0.1090 - val_loss: 16.7288 - val_acc: 0.1140
Epoch 73/100
30/30 [==============================] - 0s - loss: 16.5678 - acc: 0.1013 - val_loss: 15.7274 - val_acc: 0.1090
Epoch 74/100
